In [ ]:
import itertools, threading, time, json
from math import atan2, degrees, pi
from pathlib import Path
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display

DEFAULTS = dict(
    r=1.0, mu_x=0.0, mu_y=0.0, sigma_00=2.0, sigma_01=0.0, sigma_11=1.0, n=50
)

def generate_gaussian_equicontour(
    r: float, mu: np.ndarray, sigma: np.ndarray, n: int
) -> np.ndarray:
    angles = np.linspace(0.0, 2.0 * np.pi, n, endpoint=False)
    circle = np.stack([np.cos(angles), np.sin(angles)], axis=1)
    circle = np.vstack([circle, circle[0]])
    l_chol = np.linalg.cholesky(sigma)
    return mu + r * circle @ l_chol.T


def contour_stats(r: float, mu: np.ndarray, sigma: np.ndarray):
    eigvals, eigvecs = np.linalg.eigh(sigma)
    a, b = r * np.sqrt(eigvals[::-1])
    orient_deg = degrees(atan2(eigvecs[1, 1], eigvecs[0, 1]))
    area = pi * a * b
    return a, b, orient_deg, area


fig = go.FigureWidget()
fig.update_layout(
    title_text="Gaussian Equicontour (live)",
    xaxis=dict(range=[-5, 5], scaleanchor="y", scaleratio=1),
    yaxis=dict(range=[-5, 5]),
    height=600,
    width=600,
    showlegend=True,
    legend=dict(x=0.01, y=0.99),
)
display(fig)

r_slider = widgets.FloatSlider(
    value=DEFAULTS["r"], min=0.1, max=5.0, step=0.1, description="r"
)
n_slider = widgets.IntSlider(
    value=DEFAULTS["n"], min=5, max=200, step=5, description="n"
)
mu_x_slider = widgets.FloatSlider(
    value=DEFAULTS["mu_x"], min=-3.0, max=3.0, step=0.1, description="mu_x"
)
mu_y_slider = widgets.FloatSlider(
    value=DEFAULTS["mu_y"], min=-3.0, max=3.0, step=0.1, description="mu_y"
)
sigma_00_slider = widgets.FloatSlider(
    value=DEFAULTS["sigma_00"], min=0.1, max=5.0, step=0.1, description="sigma_00"
)
sigma_01_slider = widgets.FloatSlider(
    value=DEFAULTS["sigma_01"], min=-2.0, max=2.0, step=0.1, description="sigma_01"
)
sigma_11_slider = widgets.FloatSlider(
    value=DEFAULTS["sigma_11"], min=0.1, max=5.0, step=0.1, description="sigma_11"
)

save_btn = widgets.Button(description="Save contour", button_style="success")
clear_btn = widgets.Button(description="Clear saved", button_style="danger")
stats_out = widgets.Output(layout={"border": "1px solid gray"})

controls = widgets.VBox(
    [
        widgets.HBox([r_slider, n_slider]),
        widgets.HBox([mu_x_slider, mu_y_slider]),
        widgets.HBox([sigma_00_slider, sigma_01_slider, sigma_11_slider]),
        widgets.HBox([save_btn, clear_btn]),
        widgets.Label("Saved contour stats:"),
        stats_out,
    ]
)
display(controls)

saved_settings = []
palette = itertools.cycle(
    [
        "royalblue",
        "darkorange",
        "seagreen",
        "firebrick",
        "orchid",
        "goldenrod",
        "dodgerblue",
        "olive",
        "sienna",
        "purple",
    ]
)


def reset_sliders():
    r_slider.value = DEFAULTS["r"]
    n_slider.value = DEFAULTS["n"]
    mu_x_slider.value = DEFAULTS["mu_x"]
    mu_y_slider.value = DEFAULTS["mu_y"]
    sigma_00_slider.value = DEFAULTS["sigma_00"]
    sigma_01_slider.value = DEFAULTS["sigma_01"]
    sigma_11_slider.value = DEFAULTS["sigma_11"]


def current_contour():
    sigma = np.array(
        [
            [sigma_00_slider.value, sigma_01_slider.value],
            [sigma_01_slider.value, sigma_11_slider.value],
        ]
    )
    mu = np.array([mu_x_slider.value, mu_y_slider.value])
    try:
        np.linalg.cholesky(sigma)
    except np.linalg.LinAlgError:
        return None
    return generate_gaussian_equicontour(r_slider.value, mu, sigma, n_slider.value)


def update_live(*_):
    contour = current_contour()
    with fig.batch_update():
        if fig.data and fig.data[0].name == "live":
            fig.data[0].x = contour[:, 0] if contour is not None else []
            fig.data[0].y = contour[:, 1] if contour is not None else []
        else:
            fig.add_trace(
                go.Scatter(
                    x=contour[:, 0] if contour is not None else [],
                    y=contour[:, 1] if contour is not None else [],
                    mode="lines+markers",
                    line=dict(width=3),
                    marker=dict(size=5),
                    name="live",
                )
            )
        fig.update_layout(
            title_text=(
                "Sigma not positive-definite"
                if contour is None
                else "Gaussian Equicontour (live)"
            )
        )


for w in [
    r_slider,
    n_slider,
    mu_x_slider,
    mu_y_slider,
    sigma_00_slider,
    sigma_01_slider,
    sigma_11_slider,
]:
    w.observe(update_live, names="value")
update_live()


def refresh_stats():
    with stats_out:
        stats_out.clear_output()
        if not saved_settings:
            print("none")
            return
        header = (
            "idx  r    mu            sigma                   "
            "n   a      b      orient_deg   area"
        )
        print(header)
        print("-" * len(header))
        for i, s in enumerate(saved_settings, start=1):
            r_val = s["r"]
            mu = np.array(s["mu"])
            sigma = np.array(s["sigma"])
            a_val, b_val, orient, area_val = contour_stats(r_val, mu, sigma)
            print(
                f"{i:3d}  {r_val:3.1f}  [{mu[0]:4.1f},{mu[1]:4.1f}]  "
                f"[[{sigma[0,0]:3.1f},{sigma[0,1]:3.1f}],"
                f"[{sigma[1,0]:3.1f},{sigma[1,1]:3.1f}]]  "
                f"{s['n']:3d}  {a_val:5.2f}  {b_val:5.2f}   {orient:7.2f}  {area_val:6.2f}"
            )


def on_save(_):
    contour = current_contour()
    if contour is None:
        return
    saved_settings.append(
        {
            "r": r_slider.value,
            "mu": [mu_x_slider.value, mu_y_slider.value],
            "sigma": [
                [sigma_00_slider.value, sigma_01_slider.value],
                [sigma_01_slider.value, sigma_11_slider.value],
            ],
            "n": n_slider.value,
        }
    )
    with fig.batch_update():
        fig.add_trace(
            go.Scatter(
                x=contour[:, 0],
                y=contour[:, 1],
                mode="lines+markers",
                line=dict(color=next(palette), width=2),
                marker=dict(size=5),
                name=f"saved {len(saved_settings)}",
            )
        )
    refresh_stats()
    reset_sliders()


def on_clear(_):
    with fig.batch_update():
        fig.data = tuple([fig.data[0]])
    saved_settings.clear()
    refresh_stats()


save_btn.on_click(on_save)
clear_btn.on_click(on_clear)
refresh_stats()


def blink():
    live = fig.data[0] if fig.data else None
    if live and live.name == "live":
        live.opacity = 0.2 if (live.opacity == 1 or live.opacity is None) else 1
    global flash_timer
    flash_timer = threading.Timer(0.65, blink)
    flash_timer.start()


flash_timer = threading.Timer(0.65, blink)
flash_timer.start()